In [1]:
conda install -c conda-forge geopandas


Note: you may need to restart the kernel to use updated packages.


In [10]:
import plotly

import json
import numpy as np
import pandas as pd

import plotly.express as px

# import geopandas as gpd

In [11]:
## plotting plotly in browser

import plotly.io as pio
pio.renderers.default = 'browser'

In [12]:
print(plotly.__version__)

4.13.0


In [13]:
# print(geopandas.__version__)

In [14]:
## create geojson from shapefile 

# import geopandas

# shp_file = geopandas.read_file('myshpfile.shp')
# shp_file.to_file('myshpfile.geojson', driver='GeoJSON')

In [15]:
## add feature.id to geojson file

# see colab notebook

In [16]:
##. 1 GeoJSON
indo_states = json.load(open('indo_level1_id.geojson', 'r'))

In [17]:
indo_states_map = {}

## check if state['id'] vorrect or state['properties']['ADM1_PCODE']

# for state in indo_states['features']:
#     indo_states_map[state['properties']['ADM1_EN']] = state['id'] 

# indo_states_map

In [18]:
## 2. get covid data on province level

df = pd.read_csv('indo-cases.csv')
df.head()

## adjust names in df to match names in indo_states_map
df = df.replace('Bangka Belitung', 'Kepulauan Bangka Belitung')
df = df.replace("DKI Jakarta", 'Dki Jakarta')

## drop row 'Indonesia' 
df = df.drop(index=34)
## add column id with 'ids' from indo-state-map list
df['id'] = df['Provinsi'].apply(lambda x: indo_states_map[x])
df.head()


KeyError: 'Aceh'

In [10]:
# Step 3 : Visualization

## Option 1 : using featureidkey

fig = px.choropleth(df, geojson=indo_states, color="Kasus_Posi",
                    locations="id",
                    projection="mercator"
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [12]:
fig2 = px.choropleth_mapbox(df, geojson=indo_states, 
color= 'Kasus_Meni', locations='id',
hover_name= 'Provinsi', hover_data=['Kasus_Posi', 'Kasus_Meni'], 
title='Covid Cases in Indonesia per Province',
mapbox_style = 'carto-positron',
zoom = 3,
opacity=0.5,
)

fig2.update_geos(lataxis_showgrid=True, lonaxis_showgrid= True, fitbounds = 'locations', visible = False # set center of fig to locations of geojson and all other physical attribute(like lakes, rivers etc. to not-visible)
)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig2.show()